# Tutorial 2: Understanding Precision

**Goal**: Learn how LRS agents track and update confidence

**Time**: 15 minutes

**Prerequisites**: Tutorial 1

---

## What is Precision?

**Precision (γ)** = Agent's confidence that its world model is correct

- **γ = 0.9**: "I'm very confident I understand how tools behave"
- **γ = 0.5**: "I'm uncertain about what will happen"
- **γ = 0.2**: "I'm confused, need to explore"

**Why it matters**: Precision controls the exploration-exploitation trade-off **automatically**.

## The Math: Beta Distribution

Precision is modeled as a **Beta distribution**:

```
γ ~ Beta(α, β)
E[γ] = α / (α + β)
```

- **α (alpha)**: "Success count" - increases with low errors
- **β (beta)**: "Failure count" - increases with high errors

**Update rules**:
- Low prediction error → α += 0.1 → γ increases
- High prediction error → β += 0.2 → γ decreases

**Key property**: Loss is faster than gain (β learning rate > α learning rate)

In [ ]:
from lrs.core.precision import PrecisionParameters
import matplotlib.pyplot as plt
import numpy as np

# Create precision tracker
precision = PrecisionParameters(
    alpha=5.0,  # Initial successes
    beta=5.0,   # Initial failures
    learning_rate_gain=0.1,
    learning_rate_loss=0.2,
    threshold=0.5
)

print(f"Initial precision: {precision.value:.3f}")
print(f"Initial variance: {precision.variance:.3f}")

## Experiment 1: Successful Execution

What happens when tools consistently work?

In [ ]:
# Simulate 20 successful executions (low errors)
precision_history = [precision.value]

for i in range(20):
    error = 0.1  # Low prediction error
    precision.update(error)
    precision_history.append(precision.value)

# Plot
plt.figure(figsize=(10, 4))
plt.plot(precision_history, marker='o', linewidth=2)
plt.axhline(y=0.7, color='green', linestyle='--', label='High confidence threshold')
plt.xlabel('Step')
plt.ylabel('Precision (γ)')
plt.title('Precision Increases with Consistent Success')
plt.legend()
plt.grid(alpha=0.3)
plt.show()

print(f"Final precision: {precision.value:.3f}")
print(f"Precision increased by: {precision.value - precision_history[0]:.3f}")

## Experiment 2: Sudden Failure

What happens when a tool unexpectedly fails?

In [ ]:
# Reset
precision = PrecisionParameters(alpha=9.0, beta=1.0)  # Start with high confidence
history = [precision.value]

# Simulate sudden failure at step 10
for i in range(20):
    if i == 10:
        error = 0.95  # Massive surprise!
        print(f"⚠️  Step {i}: Unexpected failure (error={error})")
    else:
        error = 0.1   # Normal operation
    
    precision.update(error)
    history.append(precision.value)

# Plot
plt.figure(figsize=(10, 4))
plt.plot(history, marker='o', linewidth=2)
plt.axvline(x=10, color='red', linestyle='--', alpha=0.5, label='Failure event')
plt.axhline(y=0.4, color='orange', linestyle='--', label='Adaptation threshold')
plt.xlabel('Step')
plt.ylabel('Precision (γ)')
plt.title('Precision Collapses on Unexpected Failure')
plt.legend()
plt.grid(alpha=0.3)
plt.show()

print(f"\nPrecision before failure: {history[10]:.3f}")
print(f"Precision after failure: {history[11]:.3f}")
print(f"Drop: {history[10] - history[11]:.3f}")

## Experiment 3: Recovery

Can precision recover after a collapse?

In [ ]:
# Reset
precision = PrecisionParameters(alpha=5.0, beta=5.0)
history = []

# Phase 1: Failures (steps 0-10)
for i in range(10):
    precision.update(0.9)  # High errors
    history.append(precision.value)

print(f"After failures: γ = {precision.value:.3f}")

# Phase 2: Recovery (steps 10-30)
for i in range(20):
    precision.update(0.1)  # Low errors
    history.append(precision.value)

print(f"After recovery: γ = {precision.value:.3f}")

# Plot
plt.figure(figsize=(10, 4))
plt.plot(history, marker='o', linewidth=2)
plt.axvspan(0, 10, alpha=0.2, color='red', label='Failure phase')
plt.axvspan(10, 30, alpha=0.2, color='green', label='Recovery phase')
plt.xlabel('Step')
plt.ylabel('Precision (γ)')
plt.title('Precision Can Recover Through Consistent Performance')
plt.legend()
plt.grid(alpha=0.3)
plt.show()

## Hierarchical Precision

LRS agents track precision at **3 levels**:

1. **Abstract (Level 2)**: Long-term goals
2. **Planning (Level 1)**: Subgoal selection
3. **Execution (Level 0)**: Tool calls

Errors **propagate upward** when severe.

In [ ]:
from lrs.core.precision import HierarchicalPrecision

# Create hierarchical tracker
hp = HierarchicalPrecision(propagation_threshold=0.7)

print("Initial precision:")
for level, value in hp.get_all().items():
    print(f"  {level}: {value:.3f}")

# Small error at execution level
print("\nSmall error (0.3):")
hp.update('execution', 0.3)
for level, value in hp.get_all().items():
    print(f"  {level}: {value:.3f}")

# Large error at execution level
print("\nLarge error (0.9):")
hp.update('execution', 0.9)
for level, value in hp.get_all().items():
    print(f"  {level}: {value:.3f}")
    
print("\n⚠️ Notice: Large error propagated to planning level!")

## Interactive Demo: Policy Selection

See how precision affects which policy is chosen.

In [ ]:
from lrs.core.free_energy import precision_weighted_selection, PolicyEvaluation

# Two policies:
# 1. Exploit: High success, low info gain
# 2. Explore: Low success, high info gain

exploit_policy = PolicyEvaluation(
    epistemic_value=0.2,
    pragmatic_value=3.0,
    total_G=-2.8,  # Low G (good)
    expected_success_prob=0.9,
    components={}
)

explore_policy = PolicyEvaluation(
    epistemic_value=0.9,
    pragmatic_value=1.0,
    total_G=-0.1,  # Higher G
    expected_success_prob=0.5,
    components={}
)

policies = [exploit_policy, explore_policy]

# Test different precision levels
for precision in [0.2, 0.5, 0.9]:
    # Run 1000 trials
    selections = []
    for _ in range(1000):
        idx = precision_weighted_selection(policies, precision)
        selections.append(idx)
    
    exploit_pct = (1 - np.mean(selections)) * 100
    explore_pct = np.mean(selections) * 100
    
    print(f"\nPrecision γ = {precision:.1f}:")
    print(f"  Exploit: {exploit_pct:.1f}%")
    print(f"  Explore: {explore_pct:.1f}%")
    
print("\n📊 Key insight: Low precision → more exploration!")

## Key Takeaways

1. **Precision is Bayesian**: Updated via Beta distribution
2. **Asymmetric learning**: Lose confidence faster than gain it
3. **Hierarchical**: 3 levels with upward error propagation
4. **Automatic control**: Low precision → exploration, high precision → exploitation

## Next Steps

- **Tutorial 3**: Learn how to compose tools into complex policies
- **Tutorial 4**: Run the Chaos Scriptorium benchmark
- **Tutorial 5**: Integrate real LLMs for policy generation